# SoRAIM winter school - Speaker Extraction Workshop 
#### Hosted by Bar Ilan University (BIU)
prior knowledge: 
- Minor understanding of signal processing 
- Python


## 1. Intruduction (10 min)
<img src="media/ros4hri_ids.png" alt="h" width="700"/>

*https://wiki.ros.org/hri

### Speech processing in HRI

In the SPRING project the patients and ARI are having a verbal dialogue. Therefore the ability to understand the patient is key.
When failing to understand the patient, ARI could respond in a completely wrong matter,which can damage the expectance of ARI by the patient.
Scene the patient's speech might be overlapped with another human's voice (doctors,accompanying person,background speech etc.) we need to encounter and somehow get the wanted speech only.


There are two main ways of doing such task:
 1. "Blind" source separation - splits the overlapping speech into two channels (one for each speaker) with no prior knowledge about the speakers.
 2. Speaker extraction - extract the desired speaker from the overlapping speech utilizing a reference audio signal thereof.

Today we will focus on speaker extraction.
## Lets see an example




In [ ]:
#%%capture --no-display
# installing requierments
! pip3 install requests ffmpeg-python
! pip3 install pip install "deepgram-sdk>=3.*,<4.0"
! pip3 install -r requirements.txt
import gdown
id = "19WkOv8XQ4lQwtPQLp5TWMY_xDAcs0ArW"
output = 'speaker_extraction/epoch=374,val_loss=-12.62.pth'
gdown.download(id=id,output=output)
from workshop_utils import *

In [ ]:
import soundfile as sf
import jiwer
mixed,sr = sf.read('example/mix.wav')
mixed_trns = transcribe_wav('example/mix.wav')
truth ="Hello, robot. I would like to know where is room number 55, I have an appointment in 5 minutes, and I don't want to be late." 
wer_before = jiwer.wer(truth,mixed_trns)
print(f"Mixture transctiprion WER={wer_before}:\n{mixed_trns}")
display_audio(mixed,sr)
print('_____________________________________________')
print('The refrence:')
ref,sr = sf.read('example/male_ref.wav')
display_audio(ref,sr)
print('_____________________________________________')


from speaker_extraction.extraction_model.deliver import Extractor
extractor = Extractor()
e_wav = extractor.extract_wave('example/mix.wav','example/male_ref.wav','example/')[:int(mixed.shape[0]*(8000/sr))]
sf.write('example/extracted.wav',e_wav,8000)
extracted_trns = transcribe_wav('example/extracted.wav')
wer_after = jiwer.wer(truth,extracted_trns)
print(f"Extracted speech transctiprion, WER={wer_after}:\n{extracted_trns}")
display_audio(e_wav,8000)

## The tasks for today's workshop:
- [ ] Basics speaker extraction.
- [ ] Audio recording and feature extraction of audio data.
- [ ] Extracting your own voice from a mixture of two voices.
- [ ] Evaluating your results.

## 2. Basics of Speaker Extraction (20 minutes)

#### Target speech/speaker extraction (TSE) isolates the speech signal of a target speaker from a mixture of several speakers, with or without noises and  reverberations, using clues that identify the speaker in the mixture. Such clues might be a spatial clue indicating the direction of the target speaker, a video of the speaker’s lips, and a prerecorded enrollment utterance from which the speaker’s voice characteristics can be derived. 
<img src="media/delcroix01-3240008-large.gif" alt="extraction_clues" width="700">

- **Paragraph and image:** K. Zmolikova, M. Delcroix, T. Ochiai, K. Kinoshita, J. Cernocký, and D. Yu, ‘‘Neural target speech extraction: An overview,’’ IEEE Signal Process. Mag., vol. 0,  o. 3, pp. 8–29, May 2023 
  

#### In our approach, we propose a Siamese-Unet architecture that uses both representations. The Siamese encoders are applied in the frequency domain to infer the embedding of the noisy and reference spectra, respectively. The concatenated representations are then fed into the decoder to estimate the real and imaginary  components of the desired speaker, which are then inverse-transformed to the time-domain.

- A. Eisenberg, S. Gannot and S. E. Chazan, "Single microphone speaker extraction using unified time-frequency Siamese-Unet," 2022 30th European Signal Processing Conference (EUSIPCO), Belgrade, Serbia, 2022, pp. 762-766, doi: 10.23919/EUSIPCO55093.2022.9909545. 

## Introduction to the process of recording speech samples on laptops:
#### The audio samples in this section is from the Librispeech dataset from OpenSLR: https://www.openslr.org/11/
Audio is represented as a list of samples that correspond to the sound and time axis. 
Normalizing audio signal can be done in several ways, we will normalize in this method: 
$$
X_{norm} = \frac{0.9\cdot X }{max(abs(X))}\
$$
for example:

In [ ]:
import soundfile as sf
import torch
import numpy as np

samples,sr = sf.read('audio_samples/males/28233/5105-28233-0006.wav')
print(samples[20:])
print(f'samplerate={sr}')

# TODO: implement the following functions
# hint when mixing two audio signals you add both signals and then normalize, make sure both signals has same length before summation!




##### YOUR CODE HERE

def pad_signal_to_length():
    pass
def norm_signal(samples):
    pass

def mix_signals(wav1,wav2,sir=0): #mixes two audio signals with sir
    #Riminder! check lengths
    pass

The variable **wav** represent the audio samples and the variable **sr** represents the sampling rate of the audio.

The sampling rate corresponds to the number of samples this file has in one second.

The sampling rate is important for processing and playback as well, playing the same file with a wrong sampling rate will result in distortion of the original audio.


# <span style="color:red"> For the next parts audio will be played on your PC, please keep the volume down or use headphones </span>

In [ ]:
import IPython.display as ipd
import torch
print('Audio with correct sample rate')
def display_audio(samples,sr):
    if torch.is_tensor(samples):
        samples= samples.detach().numpy()
    ipd.display(ipd.Audio(samples,rate=sr))
display_audio(samples,sr)

### Recording audio using Audacity:

1. open Audacity
2. choose 1 input channel (mono) (A)
3. click record (B)
   
<img src="media/audacity_1.png" width="700" >

4. then export the audio using: file -> export -> export as WAV
5. save .wav file and copy the file to your working directory of this notebook.
6. load file using soundfile 


## 3. Hands-on Session #1 : Signals and Spectograms (15 minutes)
### 3.a. Recording Speech Samples
record your self saying the following phrase:
- Hello from Grenoble! Today I learned about audio processing and speech extraction, Let's see if my voice could be extracted.

Export the audio to a .wav file and then record your self introducing yourself - should be a few second long.
Export the second .wav file.




In [ ]:
#1. Load both files to two variables and play the files one after the other.
#2. play one of the files with sr != sr, how it sounds?
########## YOUR CODE HERE #######################

### 3.b. Hands-on Session: STFT domain

Now that you are familiar with digital audio, lets talk about spectrograms.

The sampled audio as a discrete-time signal that can be represented and processed in the time domain and in the frequency domain. 

Because the speech is a signal with large time variation (non-stationary signal), it is mandatory to use the STFT rather than simple fourier transform.

The STFT of the audio signal will give us the DFT (complex value) for each time frame, and by taking the absolute value of the STFT we get the spectrogram of the audio.

The spectrogram is used as an input feature to many deep learning algorithms. This way, the audio data can be thought of as an image. Among other processing methods, we can also adopt image processing techniques to analyze the audio signal and process it.


In [ ]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np

def plot_spectrogram(samples,sr,nfft=512):
    samples = samples.squeeze()
    hop_frac=4
    if not torch.is_tensor(samples):
        samples = torch.tensor(samples)
    samples_stft = torch.stft(samples,
                        n_fft=nfft,
                        hop_length=nfft//hop_frac,
                        win_length=nfft,
                        window=torch.hamming_window(nfft),
                        center=True,
                        pad_mode='reflect',
                        normalized=False,
                        onesided=None,
                        return_complex=True)
    spec = torch.abs(samples_stft)
    plt.figure(figsize = (10,10))
    plt.xlabel('Time [s]')
    plt.ylabel('Frequency [Hz]')
    plt.yticks(np.arange(0,(nfft//2)+1 ,16),(np.arange(0,(nfft//2)+1 ,16)*(sr/(2*nfft//2))).astype(np.int16))
    plt.xticks(np.arange(0,spec.shape[-1],50),np.arange(0,spec.shape[-1],50)*nfft/(hop_frac*sr))
    ax = plt.gca()
    spec_plot =ax.imshow(20*torch.log10(spec),origin='lower')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(spec_plot,cax=cax)
    cbar.set_label('Energy (dB)')
    plt.tight_layout()
    ax.set_title('Spectogram of the audio data in dB')
    plt.show()

s,sr = sf.read('speaker_extraction/outputs/mono_s1/ref1.wav')
plot_spectrogram(s,sr)

In [ ]:
#1. use this function to plot your Speech spectrogram
#2. play with the parameters: nfft, hop and win_length  - what is the efect on the spectrogram?

#### YOUR CODE HERE #######

# 4. Extraction model
The model arcitecture:

<img src="media/extraction_model.png" width="700">

Our model works with audio sampled at 8000 Hz, it will automaticly resample the audio will also return the extraacted audio in 8000 Hz.
When playing the extracted audio make sure to set SR=8000 !

TDL:
- [] load two audio files with **torchaudio** (HINT 1), one to be extracted and another one as interference (could be another workshop memeber or one from samples folder)
- [] Mix two files (make sure to have two diffrent speakers)
- [] Load a refrence file with torchaudio as well
- [] Start a speaker extration class instance (HINT 2)
- [] Use the extract_vec method to extract your voice.


In [ ]:
# Because working with pytorch we can load the audio with torchaudio directly to a torch.Tensor
import torchaudio
#HINT 1
#samples_mix, sr = torchaudio.load(path_to_wav)

#HINT 2: to use the model import it with the commands:
# from speaker_extraction.extraction_model.deliver import Extractor
# extractor = Extractor()

# Plot the spectrogram of the mixed signal and then the extracted signal, Do you see any difference?

####### YOUR CODE HERE #######


## 5. Evaluation and Discussion (10 minutes)
The Si-SDR (Scale invariant signal-to-distortion ratio) is a widely used metric in the field of audio signal processing, particularly in the context of source separation. 
$$
\text{Si-SDR} = 10 \log_{10} \left( \frac{{\| \frac{<\hat{s},s>}{<s,s>}\ s\|^2}}{{\| \frac{<\hat{s},s>}{<s,s>}\ s -\hat{s} \|^2}} \right)

$$
The Si-SDR is used measure the quality of separation between the original signal and the extracted signal with adaptation to the gain that the model might add to the extracted signal. 
We use the Soi-SDRi (improved) which is the deference between the Si-SDR of the output and the mixture, and the Si-SDR of the output and the real audio of the speaker (label).

The Si-SDR is suseptable to time shift and destortion, but is not sesative to changes in gain.
Larger Si-SDR denotes a better extraced sentence in this context.

# WER 
Word Error Rate (WER) is a common metric of the performance of a speech recognition or machine translation system. A lower WER denotes a better transciption of the audio.
$$
WER = \frac{S+I+D}{N}
$$
Where N is the number of words in the sentence,

I is the number of insertions,

D is the number of deletions, 

S is the number of substitutions.

In [ ]:
from torchmetrics.audio import SignalDistortionRatio
from speaker_extraction.extraction_model.deliver import Extractor
import torchaudio.functional as F
from workshop_utils import *
extractor = Extractor()
si_sdr = SignalDistortionRatio()

def prepare_batch(wav1_path,wav2_path,ref1_path,ref2_path,input_sr =16000):
    #returns mixed signal, ref1, ref2, org wav1 resampled to 8000, org wav2 resampled to 8000
    wav1,sr_wav1 = torchaudio.load(wav1_path)
    wav2,sr_wav2 = torchaudio.load(wav2_path)
    ref1,sr_ref1 = torchaudio.load(ref1_path)
    ref2,sr_ref2 = torchaudio.load(ref2_path)
    if (sr_ref1 + sr_ref2 +sr_wav1+sr_wav2)/4 != input_sr:
        raise(Exception('input sr does not match'))
    mixed = mix(wav1,wav2)
    wav1_res = F.resample(wav1,sr_wav1,8000)
    wav2_res = F.resample(wav2,sr_wav2,8000)
    mixed_res = F.resample(mixed,input_sr,8000)
    max_len = max(mixed_res.shape[-1],wav1_res.shape[-1],wav2_res.shape[-1])
    mixed_res = pad_to_length(mixed_res,max_len)
    wav1_res = pad_to_length(wav1_res,max_len)
    wav2_res = pad_to_length(wav2_res,max_len)
    return mixed,mixed_res,ref1,ref2,wav1_res,wav2_res

wav1_path='audio_samples/males/28233/5105-28233-0006.wav'
wav2_path='audio_samples/females/29095/4970-29095-0034.wav'
ref1_path='audio_samples/males/28233/5105-28233-0008.wav'
ref2_path='audio_samples/females/29095/4970-29095-0038.wav'

input_sr=16000
mixed,mixed_resampled,ref1,ref2,y_1,y_2 = prepare_batch(wav1_path,wav2_path,ref1_path,ref2_path,input_sr)
plot_spectrogram(mixed,16000)
sf.write('outputs/mixed_f1_m1.wav',mixed_resampled,8000)
print(f"Mixed signal:\n{transcribe_wav('outputs/mixed_f1_m1.wav')}")
display_audio(mixed,sr)

y_hat = extractor.extract_vec(mix =mixed.unsqueeze(0),sr_mix=16000,ref=ref1,sr_ref=16000)
# y_hat = pad_to_length(y_hat,mixed_resampled.shape[-1])
plot_spectrogram(y_hat,8000)
display_audio(y_hat,8000)
sf.write('outputs/y_hat_f1_m1.wav',y_hat,8000)
print(f"Extracted Signal:\n{transcribe_wav('outputs/y_hat_f1_m1.wav')}")


In [ ]:

def si_sdri(y,y_hat,mixed):
    y = y.squeeze()
    y_hat = y_hat.squeeze()
    mixed=  mixed.squeeze()
    input_sisdr =si_sdr(y_hat,mixed_resampled)
    output_sisdr = si_sdr(y_hat,y_1)
    return output_sisdr - input_sisdr

print(si_sdri(y_1,y_hat,mixed_resampled))

## 6. Conclusion (10 minutes)
## Anyone is keen to share his/hers results?

- We saw how to improve ASR dramatically with speaker extraction, when the wanted speaker is known to the robot.
  
- Additional resources for further learning. (papers, Sharon's website)
  1. Sharons website: https://sharongannot.group/
  2. papers: 
    - A. Eisenberg, S. Gannot and S. E. Chazan, "A two-stage speaker extraction algorithm under adverse acoustic conditions using a single-microphone" : https://arxiv.org/abs/2303.07072
    - A. Eisenberg, S. Gannot and S. E. Chazan, "Single microphone speaker extraction using unified time-frequency Siamese-Unet," 2022 30th European Signal Processing Conference (EUSIPCO), Belgrade, Serbia, 2022, pp. 762-766, doi: 10.23919/EUSIPCO55093.2022.9909545 : https://arxiv.org/pdf/2203.02941.pdf
    - K. Zmolikova, M. Delcroix, T. Ochiai, K. Kinoshita, J. Černocký and D. Yu, "Neural Target Speech Extraction: An overview," in IEEE Signal Processing Magazine, vol. 40, no. 3, pp. 8-29, May 2023, doi: 10.1109/MSP.2023.3240008. keywords: {Deep learning;Visualization;Targeting;Signal processing;Reverberation;Arrays;Speech processing;Noise measurement;Oral communication},:https://arxiv.org/pdf/2301.13341.pdf 
  